In [1]:
# import asyncio
import logging
import tomllib
from pathlib import Path

# import httpx
import numpy as np
import pandas as pd

# from aiolimiter import AsyncLimiter
# from selectolax.parser import HTMLParser

## Initialize

### Set Up Logging

In [2]:
logging.basicConfig(
    format="{levelname:8s} - {module} - {funcName}: {message}", style="{"
)

logger_main = logging.getLogger(__name__)
# logger_main.setLevel(logging.DEBUG)

### Load Dataset
#### Google Drive Path

In [3]:
with open(Path.cwd().parent / "secrets.toml", "rb") as f:
    secrets = tomllib.load(f)
    secrets["google_drive"]["path"] = Path(secrets["google_drive"]["path"])

#### Dataset

In [4]:
music_path = secrets["google_drive"]["path"].joinpath("BELL INVENTORY.xlsx")
music_raw = pd.read_excel(music_path)
music_raw.head()

,SEASON,KEY,STOCK #,QTY,SONG TITLE,ARRANGER,OCTAVES,LEVEL,TYPE,CHIMES,"INSTRUMENTS, VOICES, ETC…"
0,Worship,NaN,21867,1,5 HYMNS FOR BEGINNING RINGERS,ANNA PAGE,2-3,1,CHOIR,NaN,"5 SONGS: HOW FIRM A FOUNDATION, GOD IS SO GOOD..."
1,Spiritual,NaN,23785,8,5 SPIRITUALS FOR BEGINNING RINGERS,ANNA PAGE,2-3,1,CHOIR,NaN,"5 SONGS: MARY HAD A BABY, WERE YOU THERE, HE I..."
2,Worship,NaN,1387,5,5 X 5,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: BE THOU MY VISION, CAROL OF THE BELLS..."
3,Christmas,NaN,1435,6,5 X 5 FOR CHRISTMAS,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: SILENT NIGHT, PATAPAN, IN DULCI JUBIL..."
4,Christmas,NaN,1535,6,5 X 5 FOR CHRISTMAS II,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: WHAT CHILD IS THIS, O COME ALL YE FAI..."


### Clean Dataset
#### Raw Dataset Info

In [5]:
music_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   SEASON                     2927 non-null   object
 1   KEY                        1879 non-null   object
 2   STOCK #                    2899 non-null   object
 3   QTY                        2929 non-null   object
 4   SONG TITLE                 2938 non-null   object
 5   ARRANGER                   2936 non-null   object
 6   OCTAVES                    2933 non-null   object
 7   LEVEL                      1952 non-null   object
 8   TYPE                       2938 non-null   object
 9   CHIMES                     629 non-null    object
 10  INSTRUMENTS, VOICES, ETC…  875 non-null    object
dtypes: object(11)
memory usage: 252.6+ KB


In [6]:
# Keep the original dataset (`music_raw`) separate from the transformed dataset
# (`music`).
music = music_raw.copy()

#### Column Labels

In [7]:
music.columns = music.columns.str.lower()
music.columns = music.columns.str.replace(",", "")
music.columns = music.columns.str.replace(" ", "_")
music = music.rename(
    columns={"stock_#": "stock_no", "instruments_voices_etc…": "notes"}
)
music.head()

,season,key,stock_no,qty,song_title,arranger,octaves,level,type,chimes,notes
0,Worship,NaN,21867,1,5 HYMNS FOR BEGINNING RINGERS,ANNA PAGE,2-3,1,CHOIR,NaN,"5 SONGS: HOW FIRM A FOUNDATION, GOD IS SO GOOD..."
1,Spiritual,NaN,23785,8,5 SPIRITUALS FOR BEGINNING RINGERS,ANNA PAGE,2-3,1,CHOIR,NaN,"5 SONGS: MARY HAD A BABY, WERE YOU THERE, HE I..."
2,Worship,NaN,1387,5,5 X 5,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: BE THOU MY VISION, CAROL OF THE BELLS..."
3,Christmas,NaN,1435,6,5 X 5 FOR CHRISTMAS,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: SILENT NIGHT, PATAPAN, IN DULCI JUBIL..."
4,Christmas,NaN,1535,6,5 X 5 FOR CHRISTMAS II,DOUGLAS WAGNER,3,NaN,QUINTET,NaN,"5 SONGS: WHAT CHILD IS THIS, O COME ALL YE FAI..."


#### Convert String Columns to `StringDtype`

In [8]:
music_cols = music.columns.values
str_cols = music_cols[~np.isin(music_cols, ["qty"])]
music[str_cols] = music[str_cols].astype("string")

In [9]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   season      2927 non-null   string
 1   key         1879 non-null   string
 2   stock_no    2899 non-null   string
 3   qty         2929 non-null   object
 4   song_title  2938 non-null   string
 5   arranger    2936 non-null   string
 6   octaves     2933 non-null   string
 7   level       1952 non-null   string
 8   type        2938 non-null   string
 9   chimes      629 non-null    string
 10  notes       875 non-null    string
dtypes: object(1), string(10)
memory usage: 252.6+ KB


#### Convert Integer Column to `Int8Dtype`

In [10]:
music["qty"] = pd.to_numeric(music["qty"], errors="coerce")
music["qty"] = music["qty"].astype("Int8")

In [11]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   season      2927 non-null   string
 1   key         1879 non-null   string
 2   stock_no    2899 non-null   string
 3   qty         2928 non-null   Int8  
 4   song_title  2938 non-null   string
 5   arranger    2936 non-null   string
 6   octaves     2933 non-null   string
 7   level       1952 non-null   string
 8   type        2938 non-null   string
 9   chimes      629 non-null    string
 10  notes       875 non-null    string
dtypes: Int8(1), string(10)
memory usage: 235.4 KB


#### `season` Column

In [12]:
music["season"].unique()

<StringArray>
[    'Worship',   'Spiritual',   'Christmas',     'Wedding',     'Secular',
   'Patriotic',        'Lent',      'Easter',      'Advent',          <NA>,
       'Movie',       'Radio', 'Palm Sunday']
Length: 13, dtype: string

#### Convert `key` Column to a `location` Column

In [13]:
music["key"].value_counts()

key
Sample              1835
Secular               35
Samples                3
Slav                   2
Baptism                1
mellow dissonant       1
Hymnal                 1
Treble-only            1
Name: count, dtype: Int64

In [16]:
samples_rows = music[music["key"] == "Samples"].index.values

In [18]:
music.loc[music["key"] == "Samples", "key"] = "Sample"

In [19]:
music["key"].value_counts()

key
Sample              1838
Secular               35
Slav                   2
Baptism                1
mellow dissonant       1
Hymnal                 1
Treble-only            1
Name: count, dtype: Int64

In [20]:
music.loc[samples_rows]

,season,key,stock_no,qty,song_title,arranger,octaves,level,type,chimes,notes
2646,Worship,Sample,HB348,1,THREE HYMNS,MICHAEL HELMAN,2-3,2+,CHOIR,<NA>,"3 SONGS: HYFRYDOL, THE GOOD SHEPHERD, VARIATIO..."
2647,Worship,Sample,HB353,1,THREE HYMNS,MICHAEL HELMAN,3-5,2+,CHOIR,<NA>,"3 SONGS: HYFRYDOL, THE GOOD SHEPHERD, VARIATIO..."
2648,Easter,Sample,HB362,1,THREE HYMNS FOR EASTER,MICHAEL HELMAN,2-3,3,CHOIR,<NA>,"3 SONGS: PALM SUNDAY PRELUDE, NOW THE GREEN BL..."


#### Convert `Secular` Values into a `Secular` Boolean Column

In [25]:
season_secular = music["season"] == "Secular"
key_secular = music["key"] == "Secular"
secular_index = pd.Index(zip(season_secular, key_secular))

In [29]:
key_secular.unique()

<BooleanArray>
[<NA>, False, True]
Length: 3, dtype: boolean

In [27]:
secular_index.unique()

MultiIndex([(False,   nan),
            ( True,   nan),
            (False, False),
            (False,  True),
            (  nan, False),
            ( True, False),
            (  nan,   nan)],
           )

In [15]:
music[music["stock_no"].isnull()]

,season,key,stock_no,qty,song_title,arranger,octaves,level,type,chimes,notes
97,Worship,<NA>,<NA>,8,ACROSS THE LANDS,KEITH GETTY,5,<NA>,CHOIR,<NA>,HYMN
144,Easter,<NA>,<NA>,8,ALL HAIL THE POWER OF JESUS' NAME,KEN HOLLAND,4,<NA>,CHOIR,<NA>,<NA>
224,Christmas,<NA>,<NA>,8,ANGEL'S ANNOUNCEMENT,SANDRA EITHUN,3-5,<NA>,CHOIR,<NA>,NARRATION
255,Christmas,<NA>,<NA>,8,ANTIOCH-JOY TO THE WORLD,DOUGLAS WAGNER,3,<NA>,CHOIR,<NA>,<NA>
298,Christmas,<NA>,<NA>,6,AWAY IN THE MANGER,LANNY ALLEN,3,<NA>,CHOIR,<NA>,"ORGAN, FLUTE, TRUMPET"
346,Radio,<NA>,<NA>,1,BEATLES BEST OF,DOUGLAS WAGNER,3-5,<NA>,CHOIR,<NA>,"8 SONGS: MICHELLE, YESTERDAY, I WANT TO HOLD Y..."
491,Christmas,Sample,<NA>,1,CAROL OF THE BIRDS,CARL WILTSE,2,<NA>,CHOIR,<NA>,<NA>
592,Christmas,Sample,<NA>,1,CHRISTMAS PIPES,CARL WILTSE,6,<NA>,CHOIR,6,<NA>
593,Christmas,<NA>,<NA>,1,CHRISTMAS POPS VOL 1,DOUGLAS WAGNER,2-3,E,CHOIR,<NA>,"10 SONGS: LITTLE DRUMMER BOY, RUDOLPH, GRANDMA..."
594,Christmas,<NA>,<NA>,1,CHRISTMAS POPS VOL 2,DOUGLAS WAGNER,2-4,E,CHOIR,<NA>,"10 SONGS: I HEARD THE BELLS ON CHRISTMAS DAY, ..."


In [16]:
music[music["arranger"].isnull()]

,season,key,stock_no,qty,song_title,arranger,octaves,level,type,chimes,notes
415,Wedding,<NA>,,8,BRIDAL MARCH,<NA>,3-4,<NA>,CHOIR,<NA>,<NA>
1209,Easter,Hymnal,<NA>,8,HOSANNA! LOUD HOSANNA,<NA>,3,<NA>,CHOIR,<NA>,WITH CHOIR & CONGREGATION ON EASTER


In [49]:
music.shape

(2938, 11)

#### Create Browser Instance

## Extract

### Main Scraping Function